In [ ]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from StringModel import StringModel
from Recorder import Recorder
from tqdm import tqdm

# Define the model

### Modeling
From the wave equation with damping
$$
\frac{\partial^2 y(x,t)}{\partial t^2}+b \frac{\partial y(x,t)}{\partial t}=c^2 \frac{\partial^2 y(x,t)}{\partial x^2}
$$
where
- $y(x,t)$: displacement in y direction
- $t$: time
- $x$: displacement in x direction
- $b$: damping coefficient
- $c$: wave speed

using finite difference method (FDM), we can discretize the equation as
$$
u_i^{n+1}=\left(1+\frac{1}{2} b \Delta t\right)^{-1}\left(\left(\frac{1}{2} b \Delta t-1\right) u_i^{n-1}+2 u_i^n+C^2\left(u_{i+1}^n-2 u_i^n+u_{i-1}^n\right)\right).
$$
(explicit method)

And the stability criterion is:
$$
\frac{c\Delta t}{\Delta x} \leq 1
$$

### Initial Condition (ICs)
define a initial condition $y0$ of a string, can be any shapes.

### Boundary Condition (BCs)
we define the edges of the string as fixed, i.e. the displacement is zero at the $y(x=0)$ and  $y(x=L)$ all the time.
you can adjust the BCs in the `StringModel.step`.

```python
class StringModel():

    #...

    def step(self):

        #...

        # BCs
        self.y[[0,1,-2,-1]] = self.y0[[0,1,-2,-1]] # adjust here!


```

### Disturbancer
you can add 3 types of disturbancer to the system
- "sine disturbancer": make a point in the string oscillate with a sine wave. Use the `model.set_sine_disturbancer` method to add it to the system.
- "hammer attack": given a point in the string, hit it with a hammer. Use the `model.set_hammer_attack` method to add it to the system.
- "noise disturbancer": add noise to the string. Use the `model.set_noise_disturbancer` method to add it to the system.


In [ ]:
### model parameters ###
L = 50 # m
c = 40 # m/s
b = 0.1 # damping coefficient
dt = 0.001 # s
x = np.linspace(0, L, 256)

### set initial condition (ICs) ###

# no displacement
y0 = np.zeros_like(x)

# sine wave
# d0 = 0.1
# y0 = d0 * np.sin(2*np.pi*(x)/L)

# triangluar displacement (simluating the guitar pluck)
# d0 = 0.1
# h = 0.1
# y0[np.where(x <= d0*x[-1])] = h/(d0*x[-1]) * x[np.where(x <= d0*x[-1])]
# y0[np.where(x > d0*x[-1])] = -h/((1-d0)*x[-1]) * (x[np.where(x > d0*x[-1])] - x[-1])

### modeling ###
model = StringModel(x, y0, dt, c, L, b=b)

### set disturbance ###

# sine wave disturbance at some position
model.set_sine_disturbancer(0.04, 1/10, 'resonant')

# noise disturbance
# model.set_noise_disturbancer(1e-5)

# hammer attack
# model.set_hammer_attack(50, 0.7, 0.01, 10)


### test (ensure the model is working) ###
model.test(plot=True)

### Animation

In [ ]:
# add `%matplotlib widget` to the first line of the "file" (not this block) if using jupyter notebook in vscode.
fps = 30

fig, ax = plt.subplots(figsize=(8, 4))
line, =  ax.plot(x, y0)
text = ax.text(0.1, 0.9, f"t = {0}", transform=ax.transAxes)

ax.axhline(y=0, alpha=0.3, color="black", ls="-.")

ax.set_ylim([-1.1*0.1, 1.1*0.1])

ax.set(xlabel="x", ylabel="Displacement")
ax.set_xlim([x[0], x[-1]])

fig.tight_layout()


def animate(i):
    for _ in range(10):
        model.step()
        line.set_ydata(model.get_y())
        text.set_text(f"t = {model.time():.2f}")

    return line, text

ani = FuncAnimation(fig, animate, np.arange(1, 1000), interval=1/fps, blit=True)
# ani.save('./anime.gif', writer='pillow', fps=30)

# Recording
Simulating a omni-directional mic recording the sound of the string at the position `(d0, dist)`.
The distance between the particle i and the mic is $\sqrt{(x_i-d_0)^2 + dist^2}$. So the sound pressure the mic received is vary with different position of the particle.
Anyway, you can see the implementation in `Recorder.step_record`.

In [ ]:
L = 1
c = 427
b = 10
fs = 48000
dt = 1/fs/10
x = np.linspace(0, L, 256)

# initial condition
d0 = 0.4
y0 = np.zeros_like(x)

# sine wave
#d0 * np.sin(2*np.pi*(x)/L)

# triangle (simulating a guitar pluck)
# h = 1
# d0 = 0.4
# y0[np.where(x <= d0*x[-1])] = h/(d0*x[-1]) * x[np.where(x <= d0*x[-1])]
# y0[np.where(x > d0*x[-1])] = -h/((1-d0)*x[-1]) * (x[np.where(x > d0*x[-1])] - x[-1])

# modeling
model = StringModel(x, y0, dt, c, L, b=b)

# model.set_sine_disturbancer(0.1, 1/10, 'resonant')
model.set_hammer_attack(1e5, 0.5, 0.01, 50)
# model.set_noise_disturbancer(1e-5)


# test
model.test(plot=True)

recorder = Recorder(model, fs, 0.2, 1.5)
duration = 1 # s
num_steps = int(duration/dt)
data = np.zeros(num_steps // int(1/dt/fs))

for i in tqdm(range(num_steps)):
    model.step()
    if (i % int(1/dt/fs) == 0):
        recorder.step_record()

data = recorder.export()

# plot data
fig, ax = plt.subplots()
ax.plot(data)
ax.set(xlabel="Time", ylabel="Displacement")
fig.tight_layout()

plt.show()

In [ ]:
# save data as wav

import soundfile as sf
amp = 10
sf.write('output.wav', np.array(data * amp), fs)

# Analysis

In [ ]:
# fft
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

data_trim = data
N = len(data_trim)
T = 1/fs
buffer_size = 2048
yf = np.zeros(buffer_size)

for i in range(int(N/buffer_size)):
    yf += np.abs(fft(data_trim[i*buffer_size:(i+1)*buffer_size] * np.hanning(buffer_size))) / (N/buffer_size)

xf = fftfreq(buffer_size, T)[:buffer_size//2]
yf = 2.0/buffer_size * yf[:buffer_size//2]

fig, ax = plt.subplots()
ax.plot(xf[:200], yf[:200])

peaks, _ = find_peaks(yf[:1000], height=0.0001)
peaks = peaks * (fs/buffer_size)